In [25]:
import pandas as pd
import numpy as np

In [79]:
from google.cloud import vision
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="APAD-DARASIMI-8c1a66d445ac.json"
client = vision.ImageAnnotatorClient()
image = vision.types.Image()

In [2]:
data = pd.read_csv("output.csv")
data.head()

,type,media_url,caption,likes,comments
0,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @stevewinterphoto | One of the lucky ...,256591,1271
1,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Charlie Hamilton James @chamiltonjame...,587042,1692
2,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @babaktafreshi | A bird of freedom fl...,312169,3039
3,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Sara Hylton @sarahyltonphoto | Women ...,153079,394
4,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Ian Teh @iantehphotography | Palm oil...,163279,618


## Normalize Likes & Comments

In [3]:
Likes = data['likes']
Norm_Likes = (Likes-Likes.min())/(Likes.max()-Likes.min())
print(Norm_Likes.head())
print("The range of Norm_Likes is between %i and %i"%(Norm_Likes.max(),Norm_Likes.min()))
print()
Comments = data['comments']
Norm_Comments = (Comments-Comments.min())/(Comments.max()-Comments.min())
print(Norm_Comments.head())
print("The range of Norm_Comments is between %i and %i"%(Norm_Comments.max(),Norm_Comments.min()))

0    0.243893
1    0.996790
2    0.370521
3    0.008052
4    0.031291
Name: likes, dtype: float64
The range of Norm_Likes is between 1 and 0

0    0.350037
1    0.504023
2    0.996708
3    0.029261
4    0.111192
Name: comments, dtype: float64
The range of Norm_Comments is between 1 and 0


## Engagement Score

In [4]:
Engagement_Score = 0.4*Norm_Likes + 0.6*Norm_Comments
print(Engagement_Score.head())
Engagement_Median = Engagement_Score.median()
print("The Engagement_Median is %f"% Engagement_Median)

0    0.307579
1    0.701130
2    0.746233
3    0.020777
4    0.079232
dtype: float64
The Engagement_Median is 0.236224


## Determine Engagement [ 1 or 0 ] base on Engagement Score

In [5]:
Engagement = Engagement_Score.apply(lambda x: 1 if x >= Engagement_Median else 0)
print(Engagement.head())

0    1
1    1
2    1
3    0
4    0
dtype: int64


In [6]:
data['Norm_Likes'] = Norm_Likes
data['Norm_Comments'] = Norm_Comments
data['Engagement_Score'] = Engagement_Score
data['Engagement'] = Engagement

In [7]:
data.to_csv('TaskA.csv', index=False)

## GET IMAGE LABELS

In [80]:
def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    image.source.image_uri = uri
    response = client.label_detection(image=image)
    labels = response.label_annotations
    label_description = []
    for label in labels:
        label_description.append(label.description)
    return label_description

In [29]:
df = pd.read_csv('TaskA.csv')

In [76]:
df["image_labels"] = df["media_url"].apply(lambda x: detect_labels_uri(x))

In [83]:
df = df[['type','media_url','image_labels','caption','likes','comments','Norm_Likes','Norm_Comments','Engagement_Score','Engagement']]

In [86]:
df.to_csv('image_labels.csv', index=False)

## USE IMAGE_LABELS.CSV to get the latest csv. That should be used on the remainder sections

## LOGISTIC REGRESSION

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
lr = LogisticRegression(C=10.0, random_state=0)

x = df.iloc[:,:8]
y = df.iloc[:, -1:]
y.head()

,Engagement
0,1
1,1
2,1
3,0
4,0


In [32]:
type_image = x.iloc[:, 0].values
    
type_map = {label:idx for idx,label in 
                 enumerate(np.unique(type_image))
                }

media_url = x.iloc[:, 1].values
    
media_map = {label:idx for idx,label in 
                 enumerate(np.unique(type_image))
                }

caption = x.iloc[:, 2].values
    
caption_map = {label:idx for idx,label in 
                 enumerate(np.unique(caption))
                }

x['type'] = x['type'].map(type_map)
x['media_url'] = x['media_url'].map(media_map)
x['caption'] = x['caption'].map(caption_map)

In [35]:
x.drop(['media_url'],axis=1,inplace=True)

In [36]:
x.head()

,type,caption,likes,comments,Norm_Likes,Norm_Comments,Engagement_Score
0,0,5,256591,1271,0.243893,0.350037,0.307579
1,0,8,587042,1692,0.996790,0.504023,0.701130
2,0,0,312169,3039,0.370521,0.996708,0.746233
3,0,13,153079,394,0.008052,0.029261,0.020777
4,0,10,163279,618,0.031291,0.111192,0.079232


In [37]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state = 0)

In [38]:
lr.fit(x_train,y_train)

/Users/stevenoluwaniyi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/stevenoluwaniyi/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
y_pred = lr.predict(x_test)

In [43]:
print(confusion_matrix(y_test, y_pred))

[[52 57]
 [18 89]]


In [47]:
print("The accuracy score is ", accuracy_score(y_test, y_pred) * 100,"%" )

The accuracy score is  65.27777777777779 %


In [81]:
detect_labels_uri("https://instagram.fftw1-1.fna.fbcdn.net/v/t51.2885-15/e35/s1080x1080/74339309_354740358671306_6816845177039124307_n.jpg?_nc_ht=instagram.fftw1-1.fna.fbcdn.net&_nc_cat=1&oh=a68987b7baa84b1da348b55a7a15706c&oe=5E52C2FD")

['Bengal tiger',
 'Mammal',
 'Felidae',
 'Wildlife',
 'Tiger',
 'Terrestrial animal',
 'Big cats',
 'Snout',
 'Carnivore',
 'Siberian tiger']